In [1]:
import os, glob, re
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
DATA_DIR = "../elliptic_dataset"
WALLETS_FEATURES = "wallets_features.csv"
WALLETS_CLASSES = "wallets_classes.csv"
EDGES_PREFIX = "AddrTxAddr_edgelist_part_"

In [3]:
import os, re, glob
import pandas as pd

def load_parts(data_dir: str, base: str) -> pd.DataFrame:
    paths = glob.glob(os.path.join(data_dir, f"{base}*.csv"))
    if not paths:
        raise FileNotFoundError(f"No files found for pattern {base}_part_*.csv in {data_dir}")

    paths.sort(key=lambda p: int(re.search(r'_part_(\d+)\.csv$', p).group(1)))
    return pd.concat((pd.read_csv(p) for p in paths), ignore_index=True)

In [4]:
nodes = pd.read_csv(os.path.join(DATA_DIR, WALLETS_FEATURES))
node_labels = pd.read_csv(os.path.join(DATA_DIR, WALLETS_CLASSES))
edges_with_edge_labels = load_parts(DATA_DIR, EDGES_PREFIX)

group nodes by timesteps and see how many nodes we get at each time step

In [5]:
nodes_per_time_step = nodes.groupby('Time step')['address'].nunique()
nodes_per_time_step

Time step
1     34853
2     24847
3     20137
4     21030
5     23261
6     11754
7     21537
8     25762
9     19747
10    32763
11    21737
12    10076
13    23708
14     5573
15    14133
16     8097
17    16325
18     5566
19    10076
20    29451
21    33764
22    27413
23    13587
24    16651
25    17040
26    15949
27     8435
28     4940
29    16216
30     9089
31    14726
32    16133
33    13955
34    12940
35    22655
36    31764
37    15785
38    16824
39    11378
40    26723
41    22253
42    32531
43    24600
44    19668
45    25060
46    18184
47    21079
48    18717
49    12199
Name: address, dtype: int64

Investigate how do timesteps work - is it timestep in which the node inetracts or if it appears at one it remains there

In [6]:
address_time_step_counts = nodes.groupby('address')['Time step'].nunique()
address_time_step_counts.value_counts().sort_index()

print(f"Total unique addresses: {len(address_time_step_counts)}")
print(f"Total rows in nodes: {len(nodes)}")
print(f"\nAddresses appearing in:")
print(f"  1 time step: {(address_time_step_counts == 1).sum()}")
print(f"  2-5 time steps: {((address_time_step_counts >= 2) & (address_time_step_counts <= 5)).sum()}")
print(f"  6-10 time steps: {((address_time_step_counts >= 6) & (address_time_step_counts <= 10)).sum()}")
print(f"  >10 time steps: {(address_time_step_counts > 10).sum()}")
print(f"  Max time steps: {address_time_step_counts.max()}\n")

multi_time_step_addresses = address_time_step_counts[address_time_step_counts > 1].head(10)
for addr, count in multi_time_step_addresses.items():
    time_steps = nodes[nodes['address'] == addr]['Time step'].values
    print(f"{addr[:20]}... appears in {count} time steps: {sorted(time_steps)}")

Total unique addresses: 822942
Total rows in nodes: 1268260

Addresses appearing in:
  1 time step: 761455
  2-5 time steps: 59947
  6-10 time steps: 1315
  >10 time steps: 225
  Max time steps: 47

1111DAYXhoxZx2tsRnzi... appears in 6 time steps: [np.int64(25), np.int64(29), np.int64(39), np.int64(39), np.int64(43), np.int64(43), np.int64(47), np.int64(48)]
111khWGs3Mj7UgKT7aS6... appears in 2 time steps: [np.int64(14), np.int64(34), np.int64(34), np.int64(34), np.int64(34)]
1121A3vrYYduVPMnfS87... appears in 3 time steps: [np.int64(9), np.int64(9), np.int64(23), np.int64(42)]
112467zLZa5J6JLmvDr3... appears in 2 time steps: [np.int64(36), np.int64(37)]
1125mk1uScXXFi5MfR6g... appears in 3 time steps: [np.int64(36), np.int64(36), np.int64(36), np.int64(37), np.int64(38)]
1126nC9nY73uYEuJrvgp... appears in 2 time steps: [np.int64(19), np.int64(24)]
11287dmR4AcCWg8g8jfn... appears in 3 time steps: [np.int64(25), np.int64(38), np.int64(43)]
1128R85E7agQiFpm7N6C... appears in 2 time steps

### Building time-step graphs
At each time step we build a graph that contaisn all the nodes that have interacted with other nodes up to that time step. A graph at time $t$, let's call it $G_t$, will contain all the nodes from $G_{t-1}$, as well as all the nodes that have had their first tarnsactions at time $t$. This way we don't have to explictly model the evolving structure of the graph and can run baselines such as a normal GCN or a EvolveGCN - the targets at time $t$ will be simply weather a node makes a tarnsaction with an illict node at time $t + 1$. Then if we predict for a bunch of related nodes that they will make a tarnsaction to some illict node, that  indicates that an illict node should appear somehwere in that region. This is a bsaeline, for proper emergence we will need an approach like TGN.

In [7]:
import torch
from torch_geometric.data import Data

In [8]:
time_steps = sorted(nodes['Time step'].unique())
print(f"Total time steps: {len(time_steps)}")

Total time steps: 49


In [9]:
all_addresses = nodes['address'].unique()

# dict of adresses mapped to unique ids to index nodes in the grapj
address_to_id = {addr: idx for idx, addr in enumerate(all_addresses)}

num_nodes_total = len(all_addresses)
print(f"Total unique addresses across all time: {num_nodes_total}")

Total unique addresses across all time: 822942


In [10]:
nodes_with_labels = nodes.merge(node_labels, on='address', how='left')
nodes_with_labels.head()

,address,Time step,num_txs_as_sender,num_txs_as receiver,first_block_appeared_in,last_block_appeared_in,lifetime_in_blocks,total_txs,first_sent_block,first_received_block,...,blocks_btwn_output_txs_max,blocks_btwn_output_txs_mean,blocks_btwn_output_txs_median,num_addr_transacted_multiple,transacted_w_address_total,transacted_w_address_min,transacted_w_address_max,transacted_w_address_mean,transacted_w_address_median,class
0,111112TykSw72ztDN2WJger4cynzWYC5w,25,0.0,1.0,439586.0,439586.0,0.0,1.0,0.0,439586.0,...,0.0,0.000000,0.0,0.0,24.0,1.0,1.0,1.0,1.0,2
1,1111DAYXhoxZx2tsRnzimfozo783x1yC2,25,0.0,8.0,439589.0,485959.0,46370.0,8.0,0.0,439589.0,...,20164.0,6624.285714,8060.0,0.0,8.0,1.0,1.0,1.0,1.0,3
2,1111DAYXhoxZx2tsRnzimfozo783x1yC2,29,0.0,8.0,439589.0,485959.0,46370.0,8.0,0.0,439589.0,...,20164.0,6624.285714,8060.0,0.0,8.0,1.0,1.0,1.0,1.0,3
3,1111DAYXhoxZx2tsRnzimfozo783x1yC2,39,0.0,8.0,439589.0,485959.0,46370.0,8.0,0.0,439589.0,...,20164.0,6624.285714,8060.0,0.0,8.0,1.0,1.0,1.0,1.0,3
4,1111DAYXhoxZx2tsRnzimfozo783x1yC2,39,0.0,8.0,439589.0,485959.0,46370.0,8.0,0.0,439589.0,...,20164.0,6624.285714,8060.0,0.0,8.0,1.0,1.0,1.0,1.0,3


Let's define a helper that will extract node features for each time-step graph $G_t$. If a node reappears at multiple time steps we use the latest node features in th graph.

In [ ]:
def extract_node_features(nodes_up_to_t, active_addresses, address_to_local_id, keep_class_labels_as_features: bool = True):
    """
    Extract latest features for each active node (OPTIMIZED VERSION).
    When a node appears multiple times, use the most recent feature values.
    
    Args:
        nodes_up_to_t: DataFrame with node data up to current time step
        active_addresses: array/list of addresses active (have emerged) up to current time step
        address_to_local_id: dict mapping address -> local node ID
    
    Returns:
        torch.Tensor: node features [num_nodes, num_features]
    """

    # select only the feature columns
    # NOTE: node class should be a feature !!!
    # to predict future emrgence we can assume we will know what are the currently illict nodes!!!
    if keep_class_labels_as_features:
        feature_cols = [col for col in nodes_up_to_t.columns 
                    if col not in ['address', 'Time step']]
    else:    
        feature_cols = [col for col in nodes_up_to_t.columns 
                    if col not in ['address', 'Time step', 'class']]
    
    # use groupby to get last row per address 
    # we sort because for each node we want to keep its latest feature set in case a node reappears at multiple time steps
    nodes_sorted = nodes_up_to_t.sort_values('Time step')
    latest_per_address = nodes_sorted.groupby('address', as_index=True)[feature_cols].last()
    
    # prepare an empty array
    num_features = len(feature_cols)
    num_active_nodes = len(active_addresses)
    node_features = np.zeros((num_active_nodes, num_features))
    
    # populate features
    for addr in active_addresses:
        if addr in latest_per_address.index:
            local_id = address_to_local_id[addr]
            node_features[local_id] = latest_per_address.loc[addr].values
    
    return torch.tensor(node_features, dtype=torch.float)

Builds an edge index that specifies the edges in the graph $G_t$.

In [12]:
def build_edge_index(edges_up_to_t, address_to_local_id):
    """
    Build edge index including ALL edges (OPTIMIZED VERSION).
    
    Args:
        edges_up_to_t: DataFrame with edge data up to current time step
        address_to_local_id: dict mapping address -> local node ID
    
    Returns:
        torch.Tensor: edge_index [2, num_edges]
    """
    # filter edges where both endpoints exist
    valid_src = edges_up_to_t['input_address'].isin(address_to_local_id.keys())
    valid_dst = edges_up_to_t['output_address'].isin(address_to_local_id.keys())
    edges_valid = edges_up_to_t[valid_src & valid_dst]
    
    if len(edges_valid) == 0:
        return torch.empty((2, 0), dtype=torch.long)
    
    # map addresses to IDs using vectorized .map()
    src_ids = edges_valid['input_address'].map(address_to_local_id).values
    dst_ids = edges_valid['output_address'].map(address_to_local_id).values
    
    # stack into edge_index format [2, num_edges]
    edge_index = np.stack([src_ids, dst_ids], axis=0)
    
    return torch.tensor(edge_index, dtype=torch.long)

Helper method to prepare binary labels based on next time step data.

In [13]:
def create_labels_for_next_timestep(current_time_step, edges_df, node_labels_df, 
                                     active_addresses, address_to_local_id):
    """
    Binary labels: 1 if node transacts with illicit node at t+1, else 0 (OPTIMIZED VERSION).
    
    Args:
        current_time_step: int, current time step t
        edges_df: DataFrame with all edge data (full dataset)
        node_labels_df: DataFrame with node class labels
        active_addresses: array/list of addresses active up to current time step
        address_to_local_id: dict mapping address -> local node ID
    
    Returns:
        torch.Tensor: labels [num_nodes]
    """
    num_active = len(active_addresses)
    labels = torch.zeros(num_active, dtype=torch.long)
    
    # get edges at t+1
    edges_next = edges_df[edges_df['Time step'] == current_time_step + 1]
    if edges_next.empty:
        return labels
    
    # get illicit addresses (as set for fast lookup)
    illicit_addresses = set(node_labels_df[node_labels_df['class'] == 1]['address'].values)
    
    # vectorized operations to find nodes transacting with illicit
    # find all source addresses where destination is illicit
    illicit_dst_mask = edges_next['output_address'].isin(illicit_addresses)
    src_to_illicit = set(edges_next.loc[illicit_dst_mask, 'input_address'].values)
    
    # find all destination addresses where source is illicit
    illicit_src_mask = edges_next['input_address'].isin(illicit_addresses)
    dst_from_illicit = set(edges_next.loc[illicit_src_mask, 'output_address'].values)
    
    # combine both sets
    nodes_transacting_with_illicit = src_to_illicit | dst_from_illicit
    
    # label assignment
    for addr in active_addresses:
        if addr in address_to_local_id and addr in nodes_transacting_with_illicit:
            local_id = address_to_local_id[addr]
            labels[local_id] = 1
    
    return labels

Helper to extract node classes.

In [14]:
def extract_node_classes(active_addresses, address_to_local_id, node_labels_df):
    """
    Extract node class labels (1=illicit, 2=licit, 3=unknown) (OPTIMIZED VERSION).
    
    Args:
        active_addresses: array/list of addresses active up to current time step
        address_to_local_id: dict mapping address -> local node ID
        node_labels_df: DataFrame with node class labels
    
    Returns:
        torch.Tensor: node classes [num_nodes]
    """
    num_active = len(active_addresses)
    node_classes = np.full(num_active, 3)
    
    # create index for faster lookup
    labels_indexed = node_labels_df.set_index('address')['class']
    
    # batch lookup
    for addr in active_addresses:
        if addr in labels_indexed.index:
            local_id = address_to_local_id[addr]
            node_classes[local_id] = labels_indexed.loc[addr]
    
    return torch.tensor(node_classes, dtype=torch.long)

Main method that build a cummulative graph at time step t.

In [15]:
def build_cumulative_graph_at_timestep(current_time_step, nodes_df, edges_df, node_labels_df):
    """
    Build cumulative graph up to time step t.
    
    Args:
        current_time_step: int, current time step t
        nodes_df: DataFrame with all node data (full dataset)
        edges_df: DataFrame with all edge data (full dataset)
        node_labels_df: DataFrame with node class labels
    
    Returns:
        tuple: (Data object, address_to_local_id dict)
    """
    # get all nodes that appeared up to time t
    nodes_up_to_t = nodes_df[nodes_df['Time step'] <= current_time_step].copy()
    active_addresses = nodes_up_to_t['address'].unique()
    address_to_local_id = {addr: idx for idx, addr in enumerate(active_addresses)}
    
    # get all edges up to time t
    edges_up_to_t = edges_df[edges_df['Time step'] <= current_time_step]
    
    # build graph components
    node_features = extract_node_features(nodes_up_to_t, active_addresses, address_to_local_id)
    edge_index = build_edge_index(edges_up_to_t, address_to_local_id)
    labels = create_labels_for_next_timestep(current_time_step, edges_df, node_labels_df, active_addresses, address_to_local_id)
    node_classes = extract_node_classes(active_addresses, address_to_local_id, node_labels_df)
    
    # create PyG Data object
    data = Data(
        x=node_features,
        edge_index=edge_index,
        y=labels,
        node_class=node_classes,
        num_nodes=len(active_addresses)
    )
    
    return data, address_to_local_id

Build all the graphs for all the time-steps.

In [17]:
import torch
from torch_geometric.data import Data

time_steps = sorted(nodes['Time step'].unique())
cumulative_graphs = []
address_mappings = []

for t in time_steps[:-1]:  # skip last timestep (no t+1 for labels)
    print(f"Building graph for time step {t}...", end=" ")
    
    graph_data, addr_mapping = build_cumulative_graph_at_timestep(
        current_time_step=t,
        nodes_df=nodes_with_labels,
        edges_df=edges_with_edge_labels,
        node_labels_df=node_labels
    )
    
    cumulative_graphs.append(graph_data)
    address_mappings.append(addr_mapping)
    
    num_positive = (graph_data.y == 1).sum().item()
    print(f"Nodes: {graph_data.num_nodes}, Edges: {graph_data.edge_index.shape[1]}, "
          f"Positive: {num_positive}")

print(f"\nTotal graphs created: {len(cumulative_graphs)}")

Building graph for time step 1... Nodes: 34853, Edges: 66836, Positive: 0
Building graph for time step 2... Nodes: 59236, Edges: 199129, Positive: 0
Building graph for time step 3... Nodes: 78510, Edges: 264124, Positive: 4
Building graph for time step 4... Nodes: 98707, Edges: 331393, Positive: 2
Building graph for time step 5... Nodes: 120865, Edges: 399829, Positive: 2
Building graph for time step 6... Nodes: 131985, Edges: 436559, Positive: 8
Building graph for time step 7... Nodes: 152051, Edges: 492636, Positive: 6
Building graph for time step 8... Nodes: 176366, Edges: 578493, Positive: 13
Building graph for time step 9... Nodes: 194983, Edges: 638467, Positive: 4
Building graph for time step 10... Nodes: 220639, Edges: 701970, Positive: 8
Building graph for time step 11... Nodes: 239172, Edges: 763390, Positive: 0
Building graph for time step 12... Nodes: 248071, Edges: 789186, Positive: 8
Building graph for time step 13... Nodes: 268231, Edges: 838562, Positive: 4
Building gra

Let's see per time step hoe many node transact with illict, licit, and unknown nodes. Maybe if we can classify the unknown better (classifiers in the paper get like 96% accuracy) we can have less sparse labels.

In [16]:
illicit_addrs = set(node_labels[node_labels['class'] == 1]['address'].values)
licit_addrs = set(node_labels[node_labels['class'] == 2]['address'].values)
unknown_addrs = set(node_labels[node_labels['class'] == 3]['address'].values)

print(f"Total addresses: Illicit={len(illicit_addrs)}, Licit={len(licit_addrs)}, Unknown={len(unknown_addrs)}\n")

for t in sorted(edges_with_edge_labels['Time step'].unique())[:20]:  # First 20 time steps
    edges_t = edges_with_edge_labels[edges_with_edge_labels['Time step'] == t]
    
    # categorize edges by destination node class
    edges_to_illicit = edges_t[edges_t['output_address'].isin(illicit_addrs)]
    edges_to_licit = edges_t[edges_t['output_address'].isin(licit_addrs)]
    edges_to_unknown = edges_t[edges_t['output_address'].isin(unknown_addrs)]
    
    # count unique source nodes transacting with each class
    nodes_transacting_to_illicit = edges_to_illicit['input_address'].nunique()
    nodes_transacting_to_licit = edges_to_licit['input_address'].nunique()
    nodes_transacting_to_unknown = edges_to_unknown['input_address'].nunique()
    
    print(f"Time step {t}: {len(edges_t)} edges")
    print(f"  Nodes transacting TO illicit: {nodes_transacting_to_illicit}")
    print(f"  Nodes transacting TO licit: {nodes_transacting_to_licit}")
    print(f"  Nodes transacting TO unknown: {nodes_transacting_to_unknown}")
    print(f"  Edges TO illicit: {len(edges_to_illicit)}")
    print(f"  Edges TO licit: {len(edges_to_licit)}")
    print(f"  Edges TO unknown: {len(edges_to_unknown)}")
    print()

Total addresses: Illicit=14266, Licit=251088, Unknown=557588

Time step 1: 66836 edges
  Nodes transacting TO illicit: 59
  Nodes transacting TO licit: 8437
  Nodes transacting TO unknown: 14729
  Edges TO illicit: 96
  Edges TO licit: 31576
  Edges TO unknown: 35164

Time step 2: 132293 edges
  Nodes transacting TO illicit: 54
  Nodes transacting TO licit: 3073
  Nodes transacting TO unknown: 6058
  Edges TO illicit: 83
  Edges TO licit: 98464
  Edges TO unknown: 33746

Time step 3: 64995 edges
  Nodes transacting TO illicit: 35
  Nodes transacting TO licit: 3389
  Nodes transacting TO unknown: 10707
  Edges TO illicit: 52
  Edges TO licit: 9846
  Edges TO unknown: 55097

Time step 4: 67269 edges
  Nodes transacting TO illicit: 211
  Nodes transacting TO licit: 5777
  Nodes transacting TO unknown: 8596
  Edges TO illicit: 263
  Edges TO licit: 13671
  Edges TO unknown: 53335

Time step 5: 68436 edges
  Nodes transacting TO illicit: 39
  Nodes transacting TO licit: 3845
  Nodes transac

This is weird. Ok i suspect teh issue is that nodes that appear at some time step t mostly have only transactions at the same time step, and dont transact in later timesteps again. let's check how many nodes transact at timesteps other than the one they were created at.

In [ ]:
# for each address, find first appearance and all appearances
address_first_appearance = nodes.groupby('address')['Time step'].min()
address_all_appearances = nodes.groupby('address')['Time step'].apply(set)

# count how many appear after their first time step
transacts_after_creation = 0
only_at_creation = 0

for addr in address_first_appearance.index:
    first_ts = address_first_appearance[addr]
    all_ts = address_all_appearances[addr]
    
    # check if node appears at any time step other than first
    if len(all_ts) > 1 or (len(all_ts) == 1 and first_ts not in all_ts):
        transacts_after_creation += 1
    else:
        only_at_creation += 1

total = len(address_first_appearance)

print(f"Total unique addresses: {total:,}")
print(f"\nNodes that transact:")
print(f"  Only at creation time step: {only_at_creation:,} ({100*only_at_creation/total:.1f}%)")
print(f"  At other time steps (after creation): {transacts_after_creation:,} ({100*transacts_after_creation/total:.1f}%)")

Total unique addresses: 822,942

Nodes that transact:
  Only at creation time step: 761,455 (92.5%)
  At other time steps (after creation): 61,487 (7.5%)


That explains it. Now lets see how does it look for some node wrt to illict transactions.

In [17]:
t = 3

# Nodes in graph at t=3
nodes_up_to_t = nodes_with_labels[nodes_with_labels['Time step'] <= t]
active_at_t = set(nodes_up_to_t['address'].unique())
print(f"Nodes in graph at t={t}: {len(active_at_t)}")

# nodes that appear at t=4
nodes_at_t_plus_1 = nodes_with_labels[nodes_with_labels['Time step'] == t + 1]
nodes_appearing_at_t_plus_1 = set(nodes_at_t_plus_1['address'].unique())
print(f"Nodes appearing at t={t+1}: {len(nodes_appearing_at_t_plus_1)}")

# check how many of t=4 nodes are brand new vs. reappearing
new_nodes_at_t_plus_1 = nodes_appearing_at_t_plus_1 - active_at_t
reappearing_nodes_at_t_plus_1 = nodes_appearing_at_t_plus_1 & active_at_t
print(f"  Brand new at t={t+1}: {len(new_nodes_at_t_plus_1)}")
print(f"  Reappearing at t={t+1}: {len(reappearing_nodes_at_t_plus_1)}")

# check who transacts with illicit at t=4
edges_at_t_plus_1 = edges_with_edge_labels[edges_with_edge_labels['Time step'] == t + 1]
illicit_addrs = set(node_labels[node_labels['class'] == 1]['address'].values)

edges_to_illicit = edges_at_t_plus_1[edges_at_t_plus_1['output_address'].isin(illicit_addrs)]
edges_from_illicit = edges_at_t_plus_1[edges_at_t_plus_1['input_address'].isin(illicit_addrs)]

nodes_transacting_with_illicit = set(edges_to_illicit['input_address'].values) | set(edges_from_illicit['output_address'].values)
print(f"\nNodes transacting with illicit at t={t+1}: {len(nodes_transacting_with_illicit)}")

# ff these, check how many existed at t=3
existing_transactors = nodes_transacting_with_illicit & active_at_t
new_transactors = nodes_transacting_with_illicit - active_at_t

print(f"  Existed in graph at t={t}: {len(existing_transactors)} ← should match positive labels")
print(f"  Brand new nodes at t={t+1}: {len(new_transactors)} ← cannot predict these")


Nodes in graph at t=3: 78510
Nodes appearing at t=4: 21030
  Brand new at t=4: 20197
  Reappearing at t=4: 833

Nodes transacting with illicit at t=4: 273
  Existed in graph at t=3: 4 ← should match positive labels
  Brand new nodes at t=4: 269 ← cannot predict these


Maybe it's better to reframe the problem as - "Will an illicit node appear in my neighborhood in at most k time steps in the future?" and use graph structure at time t to predict emergence at t+1, not necessarily the transactions (edges). Maybe we should go like - if any of the nodes within a walk of 5 (k) steps will connect to an illict node at next time step, then we say label 1? And then not only look at the next time step but multiple time-steps (all of them or some window, lets say $[t+1, t+k]$? So it's like if we are in proximit of the future illict activity, then label is one.


NOTE: so like if a node a has an edge at next time step or now or in the past (t+1) or a walk of some length k that brings it to a node that makes a transaction to an illict node (has an edge) at t+1 or later time steps, we mark it as 1 - in future the neighbourhood will have illlict nodes.

NOTE: so like if a node a has an edge at next time step or now or in the past (t+1) or a walk of some length k that brings it to a node that makes a transaction to an illict node (has an edge) at t+1 or later time steps, we mark it as 1 - in future the neighbourhood will have illlict nodes.

## K-Hop Neighborhood + Multi-Horizon Labeling

The issue with direct prediction is that 92.5% of nodes only appear once, making it impossible to predict future illicit transactions for most nodes. We reframe the problem:

**New Task**: At time step t, predict if **any node in a node's k-hop undirected neighborhood** will transact with illicit nodes in the **next p time steps**.

This gives us:
- More positive labels (neighborhood aggregates signal)
- Early warning system (detect proximity to future illicit activity)
- Tunable hyperparameters: `k_hops` (neighborhood radius) and `horizon` (how many future time steps)

Default values: `k_hops=2`, `horizon=3`

In [19]:
from scipy.sparse import csr_matrix
import scipy.sparse as sp

In [ ]:
def build_undirected_adjacency_sparse(edge_index, num_nodes):
    """
    Build symmetric sparse adjacency matrix from edge_index.
    
    Args:
        edge_index: torch.Tensor [2, num_edges], directed edge list
        num_nodes: int, total number of nodes
    
    Returns:
        scipy.sparse.csr_matrix: symmetric adjacency matrix [num_nodes, num_nodes]
    """
    if edge_index.shape[1] == 0:
        return csr_matrix((num_nodes, num_nodes))
    
    # convert to numpy
    edge_index_np = edge_index.cpu().numpy()
    
    # create directed edges
    src = edge_index_np[0]
    dst = edge_index_np[1]
    
    # add reverse edges for undirected graph 
    # NOTE: we want to calculate the k-hop distance regardless of node direction for now???
    all_src = np.concatenate([src, dst])
    all_dst = np.concatenate([dst, src])
    
    # create sparse matrix (values are 1s for adjacency)
    data = np.ones(len(all_src), dtype=np.float32)
    adjacency_matrix = csr_matrix((data, (all_src, all_dst)), shape=(num_nodes, num_nodes))
    
    return adjacency_matrix

In [ ]:
def compute_k_hop_reachability(adjacency_matrix, k_hops):
    """
    Compute k-hop reachability matrix using sparse matrix powers.
    
    Args:
        adjacency_matrix: scipy.sparse.csr_matrix, adjacency matrix
        k_hops: int, neighborhood radius
    
    Returns:
        scipy.sparse.csr_matrix: boolean matrix where [i,j]=1 if j is reachable from i in ≤k hops
    """
    num_nodes = adjacency_matrix.shape[0]
    
    # start with identity (0-hop: each node reaches itself)
    reachability = sp.eye(num_nodes, format='csr')
    
    # current power of adjacency matrix
    current_power = adjacency_matrix.copy()
    
    # add A + A^2 + ... + A^k
    # this will give us esges whenever we can reach the other matrix in a k-hop walk
    for hop in range(1, k_hops + 1):
        if hop > 1:
            current_power = current_power @ adjacency_matrix
        reachability = reachability + current_power
    
    # convert to boolean matrix
    reachability = (reachability > 0).astype(np.float32)
    
    return reachability

### !!! **NOTE** / **TODO** !!!

 Maybe instead of creating 1 and 0 binary labels we would want to have the labels be the distance to the closest node within the k-hop distance that will have an illict transaction in the future? so like 0 if its this node, 1 if the neighbour, 2 if 2nd neighbor? Maybe even not limit it to the k-hop and just calculate that for every node in the graph?I think this would be more precise??? Then it becomes a regression problem - Franek

In [ ]:
def create_labels_k_hop_multi_horizon(current_time_step, edges_df, node_labels_df, 
                                       active_addresses, address_to_local_id,
                                       edge_index, num_nodes, 
                                       k_hops=2, horizon=3):
    """
    Binary labels with k-hop neighborhood + multi-horizon prediction.
    
    Label = 1 if ANY node in the k-hop neighborhood transacts with illicit 
    in time steps [t+1, t+2, ..., t+horizon]
    
    Args:
        current_time_step: int, current time step t
        edges_df: DataFrame with all edge data
        node_labels_df: DataFrame with node class labels
        active_addresses: array/list of addresses active up to t
        address_to_local_id: dict mapping address -> local node ID
        edge_index: torch.Tensor [2, num_edges], graph structure at time t
        num_nodes: int, number of nodes in graph
        k_hops: int, neighborhood radius (default=2)
        horizon: int, number of future time steps to check (default=3)
    
    Returns:
        torch.Tensor: labels [num_nodes]
    """

    # NOTE / TODO: maybe instead of creating 1 and 0 labels we would 
    # want to have the labels be the distance to the closest node
    # within the k-hop distance that will have an illict transaction in the future?
    # so like 0 if its this node, 1 if the neighbour, 2 if 2nd neighbor?
    # maybe even not limit it to the k-hop and just calculate that for every node in the graph?
    # I think this would be more precise??? - Franek

    labels = torch.zeros(num_nodes, dtype=torch.long)
    
    # build undirected adjacency matrix
    adjacency_matrix = build_undirected_adjacency_sparse(edge_index, num_nodes)
    
    # compute k-hop reachability
    reachability = compute_k_hop_reachability(adjacency_matrix, k_hops)
    
    # get illicit addresses
    illicit_addresses = set(node_labels_df[node_labels_df['class'] == 1]['address'].values)
    
    # collect all nodes that transact with illicit in this set
    # So the nodes in [t+1, ..., t+horizon]
    nodes_transacting_with_illicit_in_horizon = set()

    # le loop
    for future_t in range(current_time_step + 1, current_time_step + horizon + 1):

        # get edges in the future time step
        edges_future = edges_df[edges_df['Time step'] == future_t]
        if edges_future.empty:
            continue
        
        # find tarnsaction edges that go to illict addresses
        illicit_dst_mask = edges_future['output_address'].isin(illicit_addresses)

        # get the input addresses of thos edges
        src_to_illicit = set(edges_future.loc[illicit_dst_mask, 'input_address'].values)
        
        # find transaction edges that come from illict addresses
        illicit_src_mask = edges_future['input_address'].isin(illicit_addresses)

        # get the output addresses of those transactions
        dst_from_illicit = set(edges_future.loc[illicit_src_mask, 'output_address'].values)
        
        # collect addresses of nodes that tarnsact with illict nodes inthe future time step
        nodes_transacting_with_illicit_in_horizon.update(src_to_illicit | dst_from_illicit)
    
    # map these addresses to local IDs (only those in current graph)
    future_illicit_transactor_ids = []
    for addr in nodes_transacting_with_illicit_in_horizon:
        # if the address is in the current geaph (until time t) keep the info that it will transact with an illict node
        if addr in address_to_local_id:
            future_illicit_transactor_ids.append(address_to_local_id[addr])
    
    if len(future_illicit_transactor_ids) == 0:
        return labels
    
    # for each node, check if any of its k-hop neighbors will transact with illicit
    future_illicit_transactor_ids = np.array(future_illicit_transactor_ids)
    
    # extract reachability to these specific nodes
    # reachability[i, j] = 1 if node j is reachable from node i
    reachability_to_illicit_transactors = reachability[:, future_illicit_transactor_ids]
    
    # if any illict neighbor is reachable, label = 1
    has_illicit_in_neighborhood = (reachability_to_illicit_transactors.sum(axis=1) > 0).A1
    
    labels = torch.tensor(has_illicit_in_neighborhood, dtype=torch.long)
    
    return labels

In [ ]:
def build_cumulative_graph_at_timestep_khop(current_time_step, nodes_df, edges_df, node_labels_df,
                                             k_hops=2, horizon=3):
    """
    Build cumulative graph with k-hop + multi-horizon labeling.
    
    Args:
        current_time_step: int, current time step t
        nodes_df: DataFrame with all node data
        edges_df: DataFrame with all edge data
        node_labels_df: DataFrame with node class labels
        k_hops: int, neighborhood radius (default=2)
        horizon: int, future time steps to check (default=3)
    
    Returns:
        tuple: (Data object, address_to_local_id dict)
    """
    # get all nodes that appeared up to time t
    nodes_up_to_t = nodes_df[nodes_df['Time step'] <= current_time_step].copy()
    active_addresses = nodes_up_to_t['address'].unique()
    address_to_local_id = {addr: idx for idx, addr in enumerate(active_addresses)}
    
    # get all edges up to time t
    edges_up_to_t = edges_df[edges_df['Time step'] <= current_time_step]
    
    # build graph components
    node_features = extract_node_features(nodes_up_to_t, active_addresses, address_to_local_id)
    edge_index = build_edge_index(edges_up_to_t, address_to_local_id)
    num_nodes = len(active_addresses)
    
    # k-hop + multi-horizon labels
    labels = create_labels_k_hop_multi_horizon(
        current_time_step=current_time_step,
        edges_df=edges_df,
        node_labels_df=node_labels_df,
        active_addresses=active_addresses,
        address_to_local_id=address_to_local_id, # NOTE: those only contain until time t, so all good
        edge_index=edge_index,
        num_nodes=num_nodes,
        k_hops=k_hops,
        horizon=horizon
    )
    
    node_classes = extract_node_classes(active_addresses, address_to_local_id, node_labels_df)
    
    # create PyG Data object
    data = Data(
        x=node_features,
        edge_index=edge_index,
        y=labels,
        node_class=node_classes,
        num_nodes=num_nodes
    )
    
    return data, address_to_local_id

In [24]:
# Build all graphs with k-hop + multi-horizon labeling
# Hyperparameters
K_HOPS = 2
HORIZON = 3

cumulative_graphs_khop = []
address_mappings_khop = []

print(f"Building graphs with k_hops={K_HOPS}, horizon={HORIZON}\n")

for t in time_steps[:-HORIZON]:  # need horizon future time steps
    print(f"Building graph for time step {t}...", end=" ")
    
    graph_data, addr_mapping = build_cumulative_graph_at_timestep_khop(
        current_time_step=t,
        nodes_df=nodes_with_labels,
        edges_df=edges_with_edge_labels,
        node_labels_df=node_labels,
        k_hops=K_HOPS,
        horizon=HORIZON
    )
    
    cumulative_graphs_khop.append(graph_data)
    address_mappings_khop.append(addr_mapping)
    
    num_positive = (graph_data.y == 1).sum().item()
    print(f"Nodes: {graph_data.num_nodes}, Edges: {graph_data.edge_index.shape[1]}, "
          f"Positive: {num_positive}")

print(f"\nTotal graphs created: {len(cumulative_graphs_khop)}")

Building graphs with k_hops=2, horizon=3

Building graph for time step 1... Nodes: 34853, Edges: 66836, Positive: 1196
Building graph for time step 2... Nodes: 59236, Edges: 199129, Positive: 1684
Building graph for time step 3... Nodes: 78510, Edges: 264124, Positive: 2810
Building graph for time step 4... Nodes: 98707, Edges: 331393, Positive: 3411
Building graph for time step 5... Nodes: 120865, Edges: 399829, Positive: 2421
Building graph for time step 6... Nodes: 131985, Edges: 436559, Positive: 2427
Building graph for time step 7... Nodes: 152051, Edges: 492636, Positive: 8367
Building graph for time step 8... Nodes: 176366, Edges: 578493, Positive: 9638
Building graph for time step 9... Nodes: 194983, Edges: 638467, Positive: 11190
Building graph for time step 10... Nodes: 220639, Edges: 701970, Positive: 5192
Building graph for time step 11... Nodes: 239172, Edges: 763390, Positive: 686
Building graph for time step 12... Nodes: 248071, Edges: 789186, Positive: 10548
Building gr

Now this is what im talking abt - this is much better :)

---
#### TODOs for graph building:
- sanity check on the code, maybe id print the reachability matrix out just to see shit makes sense etc (well ist huge but like a chunk of it or sth)
- visualize the graphs obtained in this way
- consider framing it as a regression problem (i think it's better) as explained in the NOTE / TODO slightly above.

---
#### Next steps:
- train a GCN and EVolveGCn baselines
- figure out how do use a TGN
- do we ditch the discrete time steps and use block indices as time stamps for like eent absed type thing?
- or maybe make the windows smaller based on the block indices? Sth to think abt.

---

## K-Hop but make it a regression approach (the one from NOTE / TODO)

Helper for calculatig distances in the k-hop range to any other node in he graph (assuming undirected)

In [ ]:
def compute_distance_matrix(adjacency_matrix, k_hops):
    """
    Compute a distance matrix from any node in the graph to any other node.
    
    Args:
        adjacency_matrix: scipy.sparse.csr_matrix, adjacency matrix
        k_hops: int, neighborhood radius
    
    Returns:
        scipy.sparse.csr_matrix: boolean matrix where [i,j]=1 if j is reachable from i in ≤k hops
    """
    num_nodes = adjacency_matrix.shape[0]
    
    # start with identity (0-hop: each node reaches itself)
    visited = sp.eye(num_nodes, format='csr')
    distances = sp.csr_matrix((num_nodes, num_nodes))
    
    # current power of adjacency matrix
    current_power = adjacency_matrix.copy()
    
    # add A + A^2 + ... + A^k
    # this will give us edges whenever we can reach the other matrix in a k-hop walk
    for hop in range(1, k_hops + 1):
        if hop > 1:
            current_power = current_power @ adjacency_matrix

        # onlye keep the non visited values
        mask = current_power.multiply(1 - visited)

        # set distance matrix values to hop
        mask = mask.sign() * hop

        # update visited matrix
        visited = (visited + current_power).minimum(1)

        # update distances
        distances = distances + mask
    
    return distances

#### Helper for calculating distance based labels
**IMPORTANT NOTE**: if no illict transaction within k_hop distance limit, the labels is set to k-hops + 1!!!! 
(Would be weird to set it to -1 for regression, I guess we could then add a linear layer on top of our GCN and then just have it map to respective outputs but scre that for now)

In [ ]:
def create_labels_k_hop_distance_multi_horizon(current_time_step, edges_df, node_labels_df, 
                                       active_addresses, address_to_local_id,
                                       edge_index, num_nodes, 
                                       k_hops=2, horizon=3):
    """
    Distance-based labels with k-hop neighborhood + multi-horizon prediction.
    
    Label = distance to the closest node in k-hop neighborhood that transacts with illicit 
    in time steps [t+1, t+2, ..., t+horizon]. If no such node exists, label = k_hops + 1.
    
    Args:
        current_time_step: int, current time step t
        edges_df: DataFrame with all edge data
        node_labels_df: DataFrame with node class labels
        active_addresses: array/list of addresses active up to t
        address_to_local_id: dict mapping address -> local node ID
        edge_index: torch.Tensor [2, num_edges], graph structure at time t
        num_nodes: int, number of nodes in graph
        k_hops: int, neighborhood radius (default=2)
        horizon: int, number of future time steps to check (default=3)
    
    Returns:
        torch.Tensor: labels [num_nodes] in range [0, k_hops + 1]
            - 0: node itself will transact with illicit
            - 1, 2, ..., k_hops: distance to nearest future illicit transactor
            - k_hops + 1: no illicit activity in k-hop neighborhood
    """
    
    # build undirected adjacency matrix
    adjacency_matrix = build_undirected_adjacency_sparse(edge_index, num_nodes)
    
    # compute distances to neighbours within k-hops
    distances = compute_distance_matrix(adjacency_matrix, k_hops)
    
    # get illicit addresses
    illicit_addresses = set(node_labels_df[node_labels_df['class'] == 1]['address'].values)
    
    # collect all nodes that transact with illicit in this set
    # So the nodes in [t+1, ..., t+horizon]
    nodes_transacting_with_illicit_in_horizon = set()

    # le loop
    for future_t in range(current_time_step + 1, current_time_step + horizon + 1):

        # get edges in the future time step
        edges_future = edges_df[edges_df['Time step'] == future_t]
        if edges_future.empty:
            continue
        
        # find tarnsaction edges that go to illict addresses
        illicit_dst_mask = edges_future['output_address'].isin(illicit_addresses)

        # get the input addresses of thos edges
        src_to_illicit = set(edges_future.loc[illicit_dst_mask, 'input_address'].values)
        
        # find transaction edges that come from illict addresses
        illicit_src_mask = edges_future['input_address'].isin(illicit_addresses)

        # get the output addresses of those transactions
        dst_from_illicit = set(edges_future.loc[illicit_src_mask, 'output_address'].values)
        
        # collect addresses of nodes that tarnsact with illict nodes inthe future time step
        nodes_transacting_with_illicit_in_horizon.update(src_to_illicit | dst_from_illicit)
    
    # map these addresses to local IDs (only those in current graph)
    future_illicit_transactor_ids = []
    for addr in nodes_transacting_with_illicit_in_horizon:
        # if the address is in the current geaph (until time t) keep the info that it will transact with an illict node
        if addr in address_to_local_id:
            future_illicit_transactor_ids.append(address_to_local_id[addr])
    
    # Initialize all labels to k_hops + 1 (no illicit in k-hop neighborhood)
    labels = np.full(num_nodes, k_hops + 1, dtype=int)
    
    if len(future_illicit_transactor_ids) == 0:
        return torch.tensor(labels, dtype=torch.long)
    
    # for each node, check if any of its k-hop neighbors will transact with illicit
    future_illicit_transactor_ids = np.array(future_illicit_transactor_ids)

    # extract distances to these specific nodes
    # distances[i, j] = distance if node j is reachable from node i within k hops
    distances_to_illicit = distances[:, future_illicit_transactor_ids].toarray()

    # Create set for fast lookup
    future_illicit_set = set(future_illicit_transactor_ids)

    # Compute labels for each node
    for i in range(num_nodes):
        if i in future_illicit_set:
            # this node itself will transact with illicit
            labels[i] = 0
        else:
            # otherwise find minimum non-zero distance
            row_distances = distances_to_illicit[i, :]
            valid_distances = row_distances[row_distances > 0]
            if len(valid_distances) > 0:
                labels[i] = int(valid_distances.min())
            # if not found: stays at k_hops + 1 (no illicit in k-hop neighborhood)

    return torch.tensor(labels, dtype=torch.long)

The final helper - I do realise that this is an obnoxiously long function name.

In [ ]:
def build_cumulative_graph_at_timestep_distance_within_k_hops_regression(current_time_step, nodes_df, edges_df, node_labels_df,
                                             k_hops=2, horizon=3):
    """
    Build cumulative graph with k-hop + multi-horizon labeling.
    
    Args:
        current_time_step: int, current time step t
        nodes_df: DataFrame with all node data
        edges_df: DataFrame with all edge data
        node_labels_df: DataFrame with node class labels
        k_hops: int, neighborhood radius (default=2)
        horizon: int, future time steps to check (default=3)
    
    Returns:
        tuple: (Data object, address_to_local_id dict)
    """
    # get all nodes that appeared up to time t
    nodes_up_to_t = nodes_df[nodes_df['Time step'] <= current_time_step].copy()
    active_addresses = nodes_up_to_t['address'].unique()
    address_to_local_id = {addr: idx for idx, addr in enumerate(active_addresses)}
    
    # get all edges up to time t
    edges_up_to_t = edges_df[edges_df['Time step'] <= current_time_step]
    
    # build graph components
    node_features = extract_node_features(nodes_up_to_t, active_addresses, address_to_local_id)
    edge_index = build_edge_index(edges_up_to_t, address_to_local_id)
    num_nodes = len(active_addresses)
    
    # distance within k-hops + multi-horizon based labels
    labels = create_labels_k_hop_distance_multi_horizon(
        current_time_step=current_time_step,
        edges_df=edges_df,
        node_labels_df=node_labels_df,
        active_addresses=active_addresses,
        address_to_local_id=address_to_local_id, # NOTE: those only contain until time t, so all good
        edge_index=edge_index,
        num_nodes=num_nodes,
        k_hops=k_hops,
        horizon=horizon
    )
    
    node_classes = extract_node_classes(active_addresses, address_to_local_id, node_labels_df)
    
    # create PyG Data object
    data = Data(
        x=node_features,
        edge_index=edge_index,
        y=labels,
        node_class=node_classes,
        num_nodes=num_nodes
    )
    
    return data, address_to_local_id

Let's build the regression graphs.

In [ ]:
# Build all graphs with distance with k-hops + multi-horizon labeling
# Hyperparameters
K_HOPS = 2
HORIZON = 3

cumulative_graphs_khop_distance = []
address_mappings_khop_distance = []

print(f"Building graphs with k_hops={K_HOPS}, horizon={HORIZON}\n")

for t in time_steps[:-HORIZON]:  # need horizon future time steps
    print(f"Building graph for time step {t}...", end=" ")
    
    graph_data, addr_mapping = build_cumulative_graph_at_timestep_distance_within_k_hops_regression(
        current_time_step=t,
        nodes_df=nodes_with_labels,
        edges_df=edges_with_edge_labels,
        node_labels_df=node_labels,
        k_hops=K_HOPS,
        horizon=HORIZON
    )
    
    cumulative_graphs_khop_distance.append(graph_data)
    address_mappings_khop_distance.append(addr_mapping)
    
    # Count nodes at each distance
    label_counts = {}
    for dist in range(K_HOPS + 2):  # 0, 1, 2, ..., k_hops, k_hops+1
        count = (graph_data.y == dist).sum().item()
        label_counts[dist] = count
    
    # Build the output string
    counts_str = ", ".join([f"d={i}: {label_counts[i]}" for i in range(K_HOPS + 1)])
    counts_str += f", unreachable: {label_counts[K_HOPS + 1]}"
    
    print(f"Nodes: {graph_data.num_nodes}, Edges: {graph_data.edge_index.shape[1]}, "
          f"{counts_str}")

print(f"\nTotal graphs created: {len(cumulative_graphs_khop_distance)}")